In [1]:
import random
import numpy as np

In [2]:
BASE = 10
KAPPA = 9 # ~29 bits

PRECISION_INTEGRAL = 2
PRECISION_FRACTIONAL = 7
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL
BOUND = BASE**PRECISION

# Q field
Q = 6497992661811505123 # < 64 bits
Q_MAXDEGREE = 2
assert Q > BASE**(PRECISION * Q_MAXDEGREE) # supported multiplication degree (without truncation)
assert Q > 2*BOUND * BASE**KAPPA # supported kappa when in positive range 

# P field
P = 1802216888453791673313287943102424579859887305661122324585863735744776691801009887 # < 270 bits
P_MAXDEGREE = 9
assert P > Q
assert P > BASE**(PRECISION * P_MAXDEGREE)

In [3]:
def encode(rational, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = int(rational * BASE**precision_fractional)
    field_element = upscaled % field
    return field_element

def decode(field_element, field=Q, precision_fractional=PRECISION_FRACTIONAL):
    upscaled = field_element if field_element <= field/2 else field_element - field
    rational = upscaled / BASE**precision_fractional
    return rational

def share(secret, field=Q):
    first  = random.randrange(field)
    second = (secret - first) % field
    return [first, second]

def reconstruct(shares, field=Q):
    return sum(shares) % field

In [4]:
def add(x, y, field=Q):
    return [ (xi + yi) % field for xi, yi in zip(x, y) ]

def sub(x, y, field=Q):
    return [ (xi - yi) % field for xi, yi in zip(x, y) ]

def add_public(x, k, field=Q):
    return add(x, [k, 0], field)

def sub_public(x, k, field=Q):
    return sub(x, [k, 0], field)

def mul_public(x, k, field=Q):
    return [ (xi * k) % field for xi in x ]

In [5]:
def generate_multiplication_triple(field=Q):
    a = random.randrange(field)
    b = random.randrange(field)
    c = a * b % field
    return share(a, field), share(b, field), share(c, field)

def mul(x, y, triple, field=Q):
    a, b, c = triple
    # local masking
    d = sub(x, a, field)
    e = sub(y, b, field)
    # communication: the players simultanously send their shares to the other
    delta = reconstruct(d, field)
    epsilon = reconstruct(e, field)
    # local combination
    r = delta * epsilon % field
    s = mul_public(a, epsilon, field)
    t = mul_public(b, delta, field)
    return add(s, add(t, add_public(c, r, field), field), field)


In [6]:
class PrivateValue(object):
    
    def __init__(self, shares=None):
        self.shares = shares
    
    def secure(secret):
        z = PrivateValue()
        z.shares = share(encode(secret))
        return z
    
    def unwrap(self):
        return self.shares
    
    def reconstruct(self):
        return PublicValue(reconstruct(self.shares))
    
    def reveal(self):
        #return decode(reconstruct(reshare(self.shares)))
        return decode(reconstruct(self.shares))
    
    def __repr__(self):
        return "PrivateValue(%s)" % self.reveal()
    
    def __add__(x, y):
        if isinstance(y, PrivateValue):
            return PrivateValue(add(x.shares, y.shares))
        elif isinstance(y, PublicValue):
            return PrivateValue(add_public(x.shares, y.value))
        else:
            raise ValueError
    
    def __sub__(x, y):
        if isinstance(y, PrivateValue):
            return PrivateValue(sub(x.shares, y.shares))
        elif isinstance(y, PublicValue):
            return PrivateValue(sub_public(x.shares, y.value))
        else:
            raise ValueError
    
    def __mul__(x, y):
        if isinstance(y, PrivateValue):
            triple = generate_multiplication_triple()
            z = PrivateValue()
            z.shares = mul(x.shares, y.shares, triple)
            assert reconstruct(z.shares) == reconstruct(x.shares) * reconstruct(y.shares) % Q
            z.shares = truncate(z.shares)
            return z
        elif isinstance(y, PublicValue):
            return PrivateValue(mul_public(x.shares, y.value))
        else:
            raise ValueError
    
    def __pow__(x, e):
        #print("__pow__")
        z = PrivateValue.secure(1)
        for _ in range(e):
            z = z * x
        return z
    
class PublicValue(object):
    
    def __init__(self, value=None):
        self.value = value
    
    def unwrap(self):
        return self.value
    
    def share(self):
        return PrivateValue(share(self.value))
    
    def __repr__(self):
        return "PublicValue(%s)" % self.value
    
    def __add__(x, y):
        if isinstance(y, PublicValue):
            z = PublicValue()
            z.value = (x.value + y.value) % Q
            return z
        elif isinstance(y, PrivateValue):
            return y + x
        else:
            raise ValueError
            
    def __sub__(x, y):
        if isinstance(y, PublicValue):
            return PublicValue((x.value - y.value) % Q)
        else:
            raise ValueError
    
    def __mul__(x, y):
        if isinstance(y, PublicValue):
            z = PublicValue()
            z.value = (x.value * y.value) % Q
            return z
        elif isinstance(y, PrivateValue):
            return y * x
        else:
            raise ValueError
            
    def __eq__(x, y):
        if isinstance(y, PublicValue):
            return x.value == y.value
        else:
            raise ValueError
            

In [7]:
wrap = np.vectorize(lambda x: PublicValue(x))
unwrap = np.vectorize(lambda x: x.unwrap())

share_vec = np.vectorize(lambda x: x.share())
reconstruct_vec = np.vectorize(lambda x: x.reconstruct())

In [8]:
def generate_matmul_triple(m, k, n):
    r = wrap(np.random.randint(Q, size=(m, k)))
    s = wrap(np.random.randint(Q, size=(k, n)))
    t = np.dot(r, s)
    return share_vec(r), share_vec(s), share_vec(t)

# def matmul(x, y, triple):
#     r, s, t = triple
#     rho = reconstruct_vec(x - r)
#     sigma = reconstruct_vec(y - s)
#     return np.dot(rho, sigma) + np.dot(r, sigma) + np.dot(rho, s) + t

In [9]:
# using trick from SecureML paper that only requires a local operation

def truncate(x, amount=PRECISION_FRACTIONAL, field=Q):
    y0 = x[0] // BASE**amount
    y1 = field - ((field - x[1]) // BASE**amount)
    return [y0, y1]

In [10]:
encode_vec = np.vectorize(lambda x: encode(x))
decode_vec = np.vectorize(lambda x: decode(x))

In [11]:
# truncate_vec = np.vectorize(lambda x: truncate(x))

In [12]:
secure = np.vectorize(lambda x: PrivateValue.secure(x))

In [13]:
get_share = np.vectorize(lambda x, y: x.shares[y])
add_shares = np.vectorize(lambda a, b: (a + b) % Q)

In [14]:
def send_share(share, socket):
    socket.send_json(share.tolist())

def receive_share(socket):
    return np.array(socket.recv_json())

def swap_shares(share, party, socket):
    if (party == 0):
        send_share(share, socket)
        share_other = receive_share(socket)
    elif (party == 1):
        share_other = receive_share(socket)
        send_share(share, socket)
    return share_other

In [15]:
def combine_shares(share, party, socket):
    share_other = swap_shares(share, party, socket)
    return decode_vec(share + share_other)

In [16]:
add_shares_INT = np.vectorize(lambda a, b: np.add(a, b, dtype=object) % Q)
convert_to_int = np.vectorize(lambda x: int(x))

In [17]:
def generate_multiplication_triple_communication(party, socket):
    # TODO: a third party should generate these triples
    
    if (party == 0):
        triple_shares = generate_multiplication_triple()
        triple_alice = [triple_shares[0][0],triple_shares[1][0],triple_shares[2][0]]
        triple_bob = [triple_shares[0][1],triple_shares[1][1],triple_shares[2][1]]
        response = swap_shares(np.array(triple_bob), party, socket)
        return triple_alice
    elif (party == 1):
        triple_bob = swap_shares(np.array("OK"), party, socket)
        return triple_bob

In [18]:
def mul_communication(x, y, party, socket, field=Q):
    
    # TODO: compute triples offline
    triple = generate_multiplication_triple_communication(party, socket)
    a, b, c = triple
    
    # local masking
    y = convert_to_int(y)
    b = convert_to_int(b)
    d = np.subtract(x, a, dtype=object) #% field
    e = np.subtract(y, b, dtype=object) #% field
    
    # communication: the players simultanously send their shares to the other
    d_other = swap_shares(d, party, socket)
    e_other = swap_shares(e, party, socket)
    
    delta = add_shares_INT(d, d_other)
    epsilon = add_shares_INT(e, e_other)
    
    # local combination
    r = np.multiply(delta, epsilon, dtype=object) % field
    r = encode_vec(decode_vec(r) / 2)
    
    s = np.multiply(a, epsilon, dtype=object) % field
    t = np.multiply(b, delta, dtype=object) % field
    
    share = np.add(np.add(s, t, dtype=object), c, dtype=object)
    share = decode_vec(add_shares_INT(share, r))
    
    return share

In [19]:
def generate_matmul_triple_communication(m, k, n, party, socket):
    # TODO: a third party should generate these triples
    
    if (party == 0):
        triple_shares = generate_matmul_triple(m, k, n)
        triple_alice = [[],[],[]]
        triple_bob = [[],[],[]]

        for row in triple_shares[0]:
            triple_alice[0].append(get_share(row, 0))
            triple_bob[0].append(get_share(row, 1))

        for row in triple_shares[1]:
            triple_alice[1].append(get_share(row, 0))
            triple_bob[1].append(get_share(row, 1))

        for row in triple_shares[2]:
            triple_alice[2].append(get_share(row, 0))
            triple_bob[2].append(get_share(row, 1))

        for row in triple_bob:
            for i in range(len(row)):
                row[i] = row[i].tolist()

        response = swap_shares(np.array(triple_bob), party, socket)
        return triple_alice
    
    elif (party == 1):
        triple_bob = swap_shares(np.array("OK"), party, socket)
        return triple_bob

In [20]:
def matmul_communication(x, y, party, socket):
    
    # TODO: compute triples offline
    
    x_height = x.shape[0]
    x_width = x.shape[1]
    
    y_height = y.shape[0]
    y_width = y.shape[1]
    
    assert x_width == y_height
    
    triple = generate_matmul_triple_communication(x_height, x_width, y_width, party, socket)
    
    r, s, t = triple
    rho_local = x - r
    sigma_local = y - s
    
    # Communication
    rho_other = swap_shares(rho_local, party, socket)
    sigma_other = swap_shares(sigma_local, party, socket)
    
    # They both add up the shares locally
    rho = wrap(add_shares(rho_local, rho_other))
    sigma = wrap(add_shares(sigma_local, sigma_other))
    
    share = np.dot(wrap(r), sigma) + np.dot(rho, wrap(s)) + wrap(t)
    
    rs = np.dot(rho, sigma)
    rs = unwrap(rs)
    rs[:] = [x / 2 for x in rs]
    rs = wrap(rs)
    
    share = share + rs
    
    return unwrap(share)

In [21]:
# TODO: we can remove this insecure function with powering triples
def sigmoid_communication(share, sigmoid, party, socket):
    if (party == 0):
        #combined = secure(combine_shares(share, party, socket))
        #combined = sigmoid.evaluate(combined)
        combined = combine_shares(share, party, socket)
        #print("combined", combined)
        combined = sigmoid.evaluate(combined)
        #print("combined", combined)
        shares = secure(combined)
        alice = get_share(shares, 0)
        bob = get_share(shares, 1)
        swap_shares(bob, party, socket)
        return alice
    elif (party == 1):
        combine_shares(share, party, socket)
        bob = swap_shares(np.array("OK"), party, socket)
        return bob

In [22]:
class Sigmoid:
    
    def evaluate(self, x):
        return 1/(1+np.exp(-x))

    def derive(self, x):
        return x*(1-x)

In [23]:
class SigmoidInterpolated10:
    
    def __init__(self):
        ONE = PrivateValue.secure(1)
        W0  = PrivateValue.secure(0.5)
        W1  = PrivateValue.secure(0.2159198015)
        W3  = PrivateValue.secure(-0.0082176259)
        W5  = PrivateValue.secure(0.0001825597)
        W7  = PrivateValue.secure(-0.0000018848)
        W9  = PrivateValue.secure(0.0000000072)
        self.sigmoid = np.vectorize(lambda x: W0 + (x * W1) + (x**3 * W3) + (x**5 * W5) + (x**7 * W7) + (x**9 * W9))
        self.sigmoid_deriv = np.vectorize(lambda x:(ONE - x) * x)
        
    def evaluate(self, x):
        return self.sigmoid(x)

    def derive(self, x):
        return self.sigmoid_deriv(x)

In [24]:
class TwoLayerNetwork:

    def __init__(self, sigmoid, party, socket):
        self.sigmoid = sigmoid
        self.party = party
        self.socket = socket
    
    def train(self, X, y, syn0, iterations=10000, alpha=1):

        # prepare alpha value
        alpha = secure(alpha)
        
        # initial weights 
        self.synapse0 = syn0
        
        ONE = encode(0.5)
        
        # training
        for i in range(iterations):

            layer0 = X
            
            layer1 = matmul_communication(layer0, self.synapse0, self.party, self.socket)
            
            # TODO: sigmoid hack combines shares: use powering triple to make it secure
            layer1 = sigmoid_communication(layer1, self.sigmoid, self.party, self.socket)
            
            layer1_error = y - layer1
            
            one_minus_layer1 = np.subtract(ONE, layer1)
            
            layer1_derivative = mul_communication(layer1, one_minus_layer1, self.party, self.socket)
            
            layer1_delta = mul_communication(layer1_error, layer1_derivative, self.party, self.socket)
            
            # TODO: remove extra encode
            layer1_delta = encode_vec(layer1_delta)
            
            new_synapse0 = matmul_communication(layer0.T, layer1_delta, self.party, self.socket)
            
            # TODO: remove extra decode
            new_synapse0 = decode_vec(new_synapse0)

            # TODO: use +=
            self.synapse0 = np.add(self.synapse0, new_synapse0, dtype=object)
            
            # TODO: avoid converting back to int
            self.synapse0[:] = [[int(x)] for x in self.synapse0]
                        
    def print_weights(self):
        print("Layer 0 weights: \n%s" % combine_shares(self.synapse0, self.party, self.socket))